In [1]:
from mofa_empirical import *
import geemap, ee
from mpl_toolkits import mplot3d
from datetime import datetime
import ipywidgets as widgets
from ipyleaflet import WidgetControl, Marker
from os import path

In [2]:
# ****** widgets of interactive system
#year_range, crop_mask, data_type

submit = widgets.Button(description='Submit', button_style='primary', tooltip='Click me')
clear = widgets.Button(description='Clear')

style = {'description_width': 'initial'}

year_range = widgets.Dropdown(
    description='Year',
    options=[
        str(x) for x in range(2006, 2019)
    ],
    style = style,
    value='2006'
)

crop_mask = widgets.Dropdown(
    description='Maize Mask',
    options=[
        'MODIS Land Cover Type 1 (IGBP) Grasslands: dominated by herbaceous annuals (<2m)',
        'MODIS Land Cover Type 2 (UMD) Grasslands: dominated by herbaceous annuals (<2m)',
        'MODIS Land Cover Type 3 (LAI) Grassland: dominated by herbaceous annuals (<2m) including cereal croplands',
        'MODIS Land Cover Type 2 (UMD) Cropland: at least 60% of area is cultivated cropland',
        'CAU Maize classification'
    ],
    style = style,
    value='MODIS Land Cover Type 2 (UMD) Grasslands: dominated by herbaceous annuals (<2m)'
)

data_source = widgets.Dropdown(
    description='Data source',
    options=[
        'Max LAI over growing season',
        'ERA5 accummulated precipitation (m)',
        'ERA5 temperature at 2m',
        'CAU Maize Map',
        'MOFA yield statistics',        
        'Empirical Yield = MaxLAI*1500-700 (kg/ha)',
        'Maize Planting Area (ha)'
    ],
    style = style,
    value='Max LAI over growing season'
)

# *** checkbox widgets ***
checkbox_mask = widgets.Checkbox(
    value = False, 
    description = "Apply maize mask",
    style = style)

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')


In [27]:
def create_mofa_yield_map(year, withPlantingArea=False):

    df = pd.read_csv('../../Empirical-yield/Ghana-Crop-Empirical-yield/Ghana_Distirct_level_Crop_Yield_Data-upto2018_NORTHERN-ONLY.csv', sep=',', header=0)
    df = df.replace('Tamale Metro','Tamale North Sub Metro')
    df_maize = df[(df['CROP']=='MAIZE') & (df['REGION']=='NORTHERN ') & (df['YEAR']== year)]
    df_maize = df_maize.replace('Central Gonja','Gonja Central')
    df_maize = df_maize.replace('East Mamprusi','Mamprusi East')
    df_maize = df_maize.replace('Tamale Metro','Tamale North Sub Metro')
    df_maize = df_maize.replace('Yendi','Yendi Municipal')
    #print(df_maize['DISTRICT'].unique())
    
    if withPlantingArea:
        df_crop_area = pd.read_csv('Qtrend/crop_area-Jose.csv', \
                            dtype={'name_adm2': str, 'maiz_r': np.float64})\
                            .sort_values('maiz_r', ascending=False)

    
    # set a new varible with value from df_maize TO CREATE A NEW MAP LAYER
    yield_features = [] # list of ee.Features
    for i, dist in enumerate(df_maize['DISTRICT'].unique()): 
        if dist in northern_district.aggregate_array('ADM2_NAME').getInfo():
            yi = df_maize[df_maize['DISTRICT']==dist]['YIELD'].values[0]

            aoi_dist = northern_district.filter(ee.Filter.eq('ADM2_NAME', dist))\
                                        .first().geometry()
            
            new_feature = ee.Feature(aoi_dist)
            new_feature = new_feature.set({'id':i, 'yield': yi, 'ADM2_NAME': dist})
            
            if withPlantingArea:
                maiz_a = df_crop_area[df_crop_area['name_adm2']==dist]['maiz_r']
                area = maiz_a.values[0] if len(maiz_a) >0 else -1
                new_feature = new_feature.set({ 'maize_area':area})

            #print(dist, '*******', new_feature.get('yield').getInfo())
            
            ## TO DO: PRE-STORE this layer, add re use 
            
            yield_features.append( new_feature ) #aoi_dist.set('yield', yi) )
        else:
            print(dist, 'not in GAUL names')
    mofa_yield = ee.FeatureCollection(yield_features)
    #print(mofa_yield.first().getInfo())
    
    return mofa_yield

In [28]:
def create_maize_area_map(aoi=ghana_district):
    
    df_crop_area = pd.read_csv('Qtrend/crop_area-Jose.csv', dtype={'name_adm2': str, 'maiz_r': np.float64})\
                                .sort_values('maiz_r', ascending=False)

    
    # set a new varible with value from df_maize TO CREATE A NEW MAP LAYER
    features = [] # list of ee.Features
    for i, dist in enumerate(aoi.aggregate_array('ADM2_NAME').getInfo()): 
        aoi_dist = aoi.filter(ee.Filter.eq('ADM2_NAME', dist))\
                                        .first().geometry()
        if dist in df_crop_area['name_adm2'].unique():
            
            maiz_a = df_crop_area[df_crop_area['name_adm2']==dist]['maiz_r']
            area = maiz_a.values[0] if len(maiz_a) >0 else -1
        else:
            #print(dist, 'has no planting area recorded.')
            area = -1
            
        new_feature = ee.Feature(aoi_dist)
        new_feature = new_feature.set({'id':i, 'maize_area':area, 'ADM2_NAME': dist})

        features.append( new_feature ) #aoi_dist.set('yield', yi) )
        
    maize_areas = ee.FeatureCollection(features)
    return maize_areas


In [58]:
mofa_yield = create_mofa_yield_map(2006)
#print(mofa_yield.getInfo())
#print(northern_district.first().getInfo())
#outline = ee.Image().paint(northern_district, 0, 2)
print(mofa_yield.aggregate_array('yield').getInfo())
print(mofa_yield.aggregate_array('maize_area').getInfo())
print(mofa_yield.aggregate_array('ADM2_NAME').getInfo())
#print(northern_district.aggregate_array('ADM2_NAME').getInfo())


Kumbungu  not in GAUL names
[1.12, 1.12, 1.5, 1.4, 1.4, 1.56, 1.56, 1.05, 1.05, 0.98, 0.97, 1.5, 1.15, 1.15, 0.86, 0.68, 1.64, 0.8, 0.8]
[-1, 11825.2, 8566.6, 8257.6, 8257.6, 10992.8, 10992.8, -1, -1, -1, 6608.8, 8807.1, -1, 16857.6, -1, 10168.1, 5951.6, 9768, 9768]
['Gonja Central', 'West Gonja', 'Yendi Municipal', 'Nanumba North', 'Nanumba South', 'Karaga', 'Gushiegu', 'Bunkpurugu Yonyo', 'Mamprusi East', 'Savelugu Nanton', 'East Gonja', 'Tamale North Sub Metro', 'Sawla/Tuna/Kalba', 'Bole', 'Saboba', 'Tolon', 'West Mamprusi', 'Zabzugu', 'Tatale']


In [5]:
def clear_clicked(button):
    Map.clear()
    Map.addLayer(outline, {}, 'Ghana districts')
    
def submit_clicked(button):
    aoi = northern_district
    
    #Map.clear_colorbar()
    output_widget.clear_output()
    year = int(year_range.value)
    iMask = crop_mask.index if checkbox_mask.value else -1
    iData = data_source.index
        
    if iData == 0: #'Max LAI over growing season': 
        band_yearly, band_reduced = load_modis_band('Lai', year, aoi, 'max')
        data = band_reduced.clip(aoi)
        vis = colorized_vis['Lai']
        title = f'Max LAI {year}'
        legend_label = 'Max LAI over growing season'
        
    elif iData == 1: #'ERA5 accummulated precipitation (m)'
        era5_in_season = load_era_band('total_precipitation', year, aoi)
        data = era5_in_season.sum().clip(aoi) 
        vis = colorized_vis['Prec']
        title = f'Precipitation {year}'
        legend_label = 'Accummulated precipitation (m)'
        
    elif iData == 2: #'ERA5 temperature at 2m'
        era5_in_season = load_era_band('temperature_2m', year, aoi)
        accum_temp = era5_in_season.sum().clip(aoi)
        data = accum_temp.divide(6) #.subtract(273.15) # six monthly temp    
        vis = {
                'min': 295.0,
                'max': 302.0,
                'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF"] 
                }
        title = f'Temperaure {year}'
        legend_label = 'Mean temperature at 2m, Degree K'
        
    elif iData == 3: #'CAU Maize Map'
        maize_imgCol = ee.ImageCollection('users/xianda19/classification_result/2021/Ghana/maize_20210501_20211011_100percentSamples')
        data = maize_imgCol.mosaic().selfMask() 
        vis = {
                'min':0, 
                'max':1
                }
        title = "maize layer"
        
        # ---------use the next two lines if to erode S2 to MODIS resolution:-------
        #kernel = ee.Kernel.square(1, 'meters')
        #data = data.focalMin(kernel = kernel, iterations=25)
        #---------------------------------------------------------------------------
        
    elif iData == 4: #'MOFA yield statistics'
        mofa_yield = create_mofa_yield_map(year)
        data = mofa_yield.reduceToImage(properties=['yield'], reducer=ee.Reducer.first()).rename('yield')
        vis = {
                'min': 0.5,
                'max': 3.5,
                'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
                }
        title = f'MOFA Yield {year}'
        legend_label = 'MOFA Yield, 1000 kg/ha'
                  
    elif iData == 5: #'Empirical Yield = MaxLAI*1500-700 (kg/ha)'
        band_yearly, band_reduced = load_modis_band('Lai', year, aoi, 'max')
        maxlai_img = band_reduced.clip(aoi)
        data = maxlai_img.multiply(1.5).subtract(0.7).rename('yield_empirical')
        vis = {
                'min': 0.5,
                'max': 5.5,
                'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
                }
        title = f'Empirical Yield calculated from MaxLAI {year}'
        legend_label = 'Empirical Yield, 1000 kg/ha'
        
    elif iData == 6: # Planted area of maize (northern districts only)
        areas = create_maize_area_map(aoi=northern_district)
        data = areas.reduceToImage(properties=['maize_area'], reducer=ee.Reducer.first()).rename('area')
        vis = {
                'min': 6000,
                'max': 16000,
                'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
                }
        title = f'Maize Planting area'
        legend_label = 'maize area, ha' # TO-DO: double check if this is ha
    
    if checkbox_mask.value:
        if iMask == 0: #LC_Type1 Grassland
            lc_crop, lc_all = load_modis_lc('LC_Type1', year, aoi, cropclasses=[10])
        elif iMask == 1: #LC_Type2 Grassland
            lc_crop, lc_all = load_modis_lc('LC_Type2', year, aoi, cropclasses=[10])
        elif iMask == 2: #LC_Type3 Grassland
            lc_crop, lc_all = load_modis_lc('LC_Type3', year, aoi, cropclasses=[1])
        elif iMask == 3: #LC_Type2 Cropland
            lc_crop, lc_all = load_modis_lc('LC_Type2', year, aoi, cropclasses=[12])
        elif iMask == 4: # CAU
            #kernel = ee.Kernel.square(10, units='meters') 
            maize_imgCol = ee.ImageCollection('users/xianda19/classification_result/2021/Ghana/maize_20210501_20211011_100percentSamples')
            lc_crop = maize_imgCol.mosaic().selfMask()           
            #lc_crop = lc_crop.focalMin(kernel = kernel, iterations=25)
            
        data = data.mask(lc_crop)
    
    Map.addLayer(data, vis, title)
    #Map.addLayer(outline, {'palette': '000000'}, 'Northern districts')
    if iData != 3:
        Map.add_colorbar(vis, label=legend_label, layer_name=title,
                        orientation="vertical", transparent_bg=True)
    
    #previous_iData = iData
    # test prints
    #with output_widget: print(year, iMask, iData)
        

In [30]:
# ###### define the study area and corresponding maize mask ######
# --------------------------------------------------------------------------------------------------------------------------------------
# imgCol_Countries = ee.FeatureCollection("USDOS/LSIB/2017")
# study_area = imgCol_Countries.filter(ee.Filter.eq('COUNTRY_NA', 'Ghana'))
cities = ee.FeatureCollection("FAO/GAUL/2015/level2")
ghana_district = cities.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))
northern_district = ghana_district.filter(ee.Filter.eq('ADM1_NAME', 'Northern'))

outline = ee.Image().paint(northern_district, 0, 2)
# --------------------------------------------------------------------------------------------------------------------------------------

Map = geemap.Map(center=tamale_centre, zoom=8) 
Map.addLayer(outline, {'palette': '000000'}, 'Northern districts')
# Map.addLayer(maize_img, visParam, "maize_img")

#Map.add_control(output_control)
full_widgets = widgets.VBox([
  widgets.HBox([Map]),
  widgets.HBox([year_range, crop_mask, data_source], layout=widgets.Layout(height="40px")),
  widgets.HBox([checkbox_mask, submit, clear], layout=widgets.Layout(height="40px"))
])

submit.on_click(submit_clicked)
clear.on_click(clear_clicked)
#Map.addLayer(outline, {'palette': '000000'}, 'Northern districts')

display(full_widgets)


In [14]:
kernel = ee.Kernel.square(10, units='meters').getInfo()
image.focalMin(kernel = kernel, iterations=25)

{'type': 'Kernel.square',
 'radius': 10,
 'units': 'meters',
 'weights': 'Unable to print weights for meters-based kernel.'}

# Interactive Map for inspecting time series

In [8]:
maize_area = create_maize_area_map(aoi=northern_district)
maize_area_data = maize_area.reduceToImage(properties=['maize_area'], reducer=ee.Reducer.first()).rename('area')
vis = {
        'min': 6000,
        'max': 16000,
        'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
        }

In [13]:
Map1 = geemap.Map(center=farm_7021YAM, zoom=8)
Map1.addLayer(outline, {}, 'Ghana districts')

for contr in Map1.controls:
    print('*******',contr)

******* WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(ToggleButton(value=False, icon='globe', layout=Layout(height='28px', padding='0px 0px 0px 4px', width='28px'), tooltip='Search location/data'),)))
******* ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text', 'zoom_out_title'])
******* ScaleControl(options=['imperial', 'max_width', 'metric', 'position', 'update_when_idle'], position='bottomleft')
******* FullScreenControl(options=['position'])
******* MeasureControl(active_color='orange', options=['active_color', 'capture_z_index', 'completed_color', 'popup_options', 'position', 'primary_area_unit', 'primary_length_unit', 'secondary_area_unit', 'secondary_length_unit'], popup_options={'className': 'leaflet-measure-resultpopup', 'autoPanPadding': [10, 10]}, position='bottomleft', primary_length_unit='kilometers')
******* AttributionControl(options=['position', 'prefix'], position='bottomright')
******* DrawControl(circle={'sh

In [26]:
Map1 = geemap.Map(center=farm_7021YAM, zoom=8)
Map1.addLayer(outline, {}, 'Ghana districts')

Map1.addLayer(maize_area_data, vis, 'Maize Planting Area', opacity=0.7)
Map1.add_colorbar(vis, label='maize area, ha', layer_name='Maize Planting Area',
                        orientation="vertical", transparent_bg=True)
# Add an output widget to the map
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map1.add_control(output_control)

file = open("Qtrend/Qtrend_ALL_DISTRICTS.png", "rb")
image = file.read()
dummy1 = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)
image_widget = WidgetControl(widget=dummy1, position='topright')
Map1.add_control(image_widget)

def refresh_trend_map():
    Map1.clear_controls()
    Map1.add_colorbar(vis, label='maize area, ha', layer_name='Maize Planting Area',
                        orientation="vertical", transparent_bg=True)
    Map1.add_control(output_control)
    Map1.add_control(image_widget)

#orignal_contrs = Map1.controls

# Capture user interaction with the map
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map1.default_style = {'cursor': 'wait'}
        # xy = ee.Geometry.Point(latlon[::-1])     

        with output_widget:
            output_widget.clear_output()
            #Map1.remove_control(image_widget)
            #image_widget.clear_output()
            Map1.clear_controls()
                
            print(latlon)
            #click_point = ee.Geometry.Point(coords.lon, coords.lat)            
            poi = ee.Geometry.Point(latlon[::-1])
            #point = ee.Image().paint(poi, 0, 10)
            #Map1.addLayer(point)
                        
            district = northern_district.filterBounds(poi).aggregate_array('ADM2_NAME').getInfo()[0]
            print(district)
            #polygon = ee.Image().paint(**{'featureCollection': aoi_dist })
            #polygonIntersects = polygon.geometry().intersects(poi)

            #print(polygonIntersects.getInfo())
            
            short_name = ''.join(district.split(' ')).replace('/', '_')
            img_fn = f"Qtrend/Qtrend_{short_name}.png"
            if path.exists(img_fn):
                file = open(img_fn, "rb")
            else:
                file = open("Q1_dummy.jpg", 'rb')
            image = file.read()
            dummy1 = widgets.Image(
                value=image,
                format='png',
                width=300,
                height=400,
            )
            image_widget = WidgetControl(widget=dummy1, position='topright')
            
            Map1.add_control(image_widget)
            #print(Map1.controls[-1])
            
    Map1.default_style = {'cursor': 'pointer'}

Map1.on_interaction(handle_interaction)
Map1

Map(center=[9.70065, -0.54129], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…